# Pseudobulk the Heart atlas data

In [ ]:
import scanpy as sc
import pandas as pd

pd.options.display.max_columns = None

## Load the raw data

In [ ]:
# downloaded from https://www.heartcellatlas.org/ # Heart Global, raw
ad = sc.read('Global_raw.h5ad')

In [ ]:
ad.obs.region_finest = ad.obs.region_finest.astype(str)

In [ ]:
ad.obs.loc[ad.obs.region_finest == 'na', 'region_finest'] = 'SAN_unknown'
ad.obs.loc[ad.obs.region_finest == 'IVS MID LV', 'region_finest'] = 'SP IVS MID LV'
ad.obs.loc[ad.obs.region_finest == 'IVS MID RV', 'region_finest'] = 'SP IVS MID RV'

In [ ]:
ad._sanitize()

## Filtering 

In [ ]:
ad = ad[(ad.obs.cell_or_nuclei == 'Nuclei') &  (ad.obs.cell_state!='unclassified')].copy()

In [ ]:
ad.obs.cell_state = ad.obs.cell_type.astype(str) + '-' + ad.obs.cell_state.astype(str)

In [ ]:
ident_cols = ['sample_ID', 'region_finest', 'cell_state', 'cell_type']
ad.obs = ad.obs[ident_cols].copy()

In [ ]:
for c in ident_cols:
    ad.obs[c] = ad.obs[c].astype(str)

In [ ]:
ad._sanitize()

## Now the pseudobulking

In [ ]:
counts = ad.obs[ident_cols].value_counts().reset_index()
counts = counts.rename(columns={'count': 'n_cells'})

In [ ]:
ad = ad[ad.obs.merge(counts, how='left').n_cells>=10].copy()

In [ ]:
sc.pp.filter_genes(ad, min_cells=50)

In [ ]:
ad.var['gene_id'] = ad.var.index
ad.var.index = ad.var['gene_name-new']
ad.var.index.name = None

In [ ]:
counts = ad.obs[ident_cols].value_counts().reset_index()
counts = counts.rename(columns={'count': 'n_cells'})

In [ ]:
adp = sc.get.aggregate(ad, ident_cols, func='sum')

In [ ]:
adp.X = adp.layers['sum'].astype(int)

In [ ]:
del adp.layers['sum']

### Add cell counts

In [ ]:
adp.obs = adp.obs.merge(counts, how='left')

In [ ]:
adp.obs.rename(columns={'region_finest': 'region'}, inplace=True)

In [ ]:
adp = adp.copy()

## Save

In [ ]:
adp.write('heart-pseudobulk.h5ad')